In [ ]:
#importar bibliotecas
#!pip install -U scikit-learn
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression , Ridge, Lasso
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score , f1_score, precision_score, recall_score
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import  GridSearchCV , train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler ,PolynomialFeatures,normalize
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale

from pandas.plotting import autocorrelation_plot
from pandas.plotting import scatter_matrix
%matplotlib inline

In [ ]:
df = pd.read_csv("dados/df2018_2022.csv", sep=",")
df[(df['temporada']<=2021)]

In [ ]:
# Visualising distribution of data

scatter_matrix(df[['mandante_placar','visitante_placar']], figsize=(10,10))

In [ ]:
#modificando o resultado para numerico
#df.loc[(df['resultado']=="V"),'resultado'] = 1
#df.loc[(df['resultado']=="D"),'resultado'] = 2
#df.loc[(df['resultado']=="E"),'resultado'] = 3

#df.loc[(df['mv']=="m"),'mandante'] = 1
#df.loc[(df['mv']=="v"),'mandante'] = 0
#df.loc[(df['mv']=="v"),'visitante'] = 1
#df.loc[(df['mv']=="m"),'visitante'] = 0

#df['mandante'] = df['mandante'].astype(int)
#df['visitante'] = df['visitante'].astype(int)
#df['resultado'] = df['resultado'].astype(int)

In [ ]:
df.info()

In [ ]:
df.tail()

In [ ]:
# listando features categoricas e numéricas:
categorical_attributes = list(df.select_dtypes(include=['object']).columns)
numerical_attributes = list(df.select_dtypes(include=['float64', 'int64']).columns)
print('categorical_attributes:', categorical_attributes)
print('numerical_attributes:', numerical_attributes)

In [ ]:
#Deixar somente as variáveis numericas 
num_data  =df.drop(['partida_id','rodada','mandante', 'visitante', 'formacao_mandante', 'formacao_visitante', 'clube_mandante', 'clube_visitante'],axis=1)
num_data

In [ ]:
#separa as features 
features = num_data.drop(['resultado'],axis=1)
#separa as labels
Y = num_data['resultado']


print('Features')
print (features.head())

print (" ")
print ('#############')
print (" ")


print ('Labels')
print (Y.head())

In [ ]:
features.isna().sum()

In [ ]:
#Escoolhendo as melhores features com Kbest

features_list = ('mandante_placar','visitante_placar',
                 'chutes_mandante','chutes_no_alvo_mandante','posse_de_bola_mandante',
                 'passes_mandante','precisao_passes_mandante','faltas_mandante','cartao_amarelo_mandante',
                 'cartao_vermelho_mandante','impedimentos_mandante','escanteios_mandante',
                 'rodata_visitante','chutes_visitante','chutes_no_alvo_visitante','posse_de_bola_visitante',
                 'passes_visitante','precisao_passes_visitante','faltas_visitante','cartao_amarelo_visitante',
                 'cartao_vermelho_visitante','impedimentos_visitante','escanteios_visitante','temporada')

k_best_features = SelectKBest(k='all')
k_best_features.fit_transform(features, Y)
k_best_features_scores = k_best_features.scores_
raw_pairs = zip(features_list[1:], k_best_features_scores)
ordered_pairs = list(reversed(sorted(raw_pairs, key=lambda x: x[1])))

k_best_features_final = dict(ordered_pairs[:27])
best_features = k_best_features_final.keys()
print ('')
print ("Melhores features:")
print (k_best_features_final)

In [ ]:
#separa as features com base no Kbest
features  = num_data.drop(['resultado','temporada', 'cartao_amarelo_visitante', 'impedimentos_visitante', 'precisao_passes_mandante', 'cartao_amarelo_mandante', 'chutes_no_alvo_visitante', 'cartao_vermelho_mandante', 'cartao_vermelho_visitante', 'escanteios_mandante', 'faltas_mandante', 'faltas_visitante', 'chutes_visitante', 'escanteios_visitante', 'chutes_no_alvo_mandante'],axis=1)

print('Features')
print (features.head())

print ('=========')

print ('Labels')
print (Y.head())

In [ ]:
#devido ao OVERFITING, estou realizando um Low Variance Filter como tentativa de resolução
#normalize = normalize(features) 
#data_scaled = pd.DataFrame(normalize)
#data_scaled.var()

In [ ]:
#storing the variance and name of variables
#variance = data_scaled.var()
#columns = features.columns

#print(variance)

In [ ]:
#saving the names of variables having variance more than a threshold value

#variable = [ ]

#or i in range(0,len(variance)):
#    if variance[i]>=0.0: #setting the threshold as 1%
#        variable.append(columns[i-1])


#variable

#for i in range(0,len(variance)):
#    print(variance[i])

In [ ]:
# criando um novo dataframe usando as variáveis ​​acima
#new_data = features[variable]

 # primeiras cinco linhas dos novos dados
#new_data.head() 

In [ ]:
# Normalizando os dados de entrada(features)
scaler = MinMaxScaler().fit(features)
X = scaler.transform(features)

print ('Features: ',X.shape)
print (X)

In [ ]:
# Realiza a divisão dos dados de treine e teste e deixa do espaço 2650 a 3090 para validação do ml
#X_train = features_scale[:2000 ]
#X_test = features_scale[2000:2650]
#y_train = labels[:2000]
#y_test = labels[2000:2650]

# Realiza a divisão de dados de treino e teste, separando 20% para teste
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

print( len(X_train), len(y_train))

print( len(X_test), len(y_test))

print(Y.shape)

In [ ]:
#Treinando e testando os modelos
print ('LogisticRegression')

clf_LR = LogisticRegression(multi_class='multinomial',max_iter=2000)
clf_LR.fit(X_train, y_train)
pred= clf_LR.predict(X_train)

lg_acc = accuracy_score(y_train, pred)
f1=f1_score(y_train,pred,average = 'micro')
print ('Acurácia LogisticRegression:{}'.format(lg_acc))
print ('F1 Score:{}'.format(f1) )

In [ ]:
#Testando LogistRegression hyper parameters

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }

search = GridSearchCV(LogisticRegression(max_iter=1000), param_grid)

search.fit(X_train,y_train)
clf = search.best_estimator_
pred= clf.predict(X_train)
lg_acc = accuracy_score(y_train, pred)


f1=f1_score(y_train,pred,average = 'macro')

print ('Acurácia LogisticRegression:{}'.format(lg_acc))
print ('F1 Score:{}'.format(f1) )

print (clf)

In [ ]:
#Treinando e testando os modelos
print ('SVC')


clf = SVC()
clf.fit(X_train, y_train)
pred= clf.predict(X_train)

svc_acc = accuracy_score(y_train, pred)
f1=f1_score(y_train,pred, average='micro')
print ('Acurácia SVC:{}'.format(svc_acc))
print ('F1 Score:{}'.format(f1) )

In [ ]:
#Testando SVC hyper parameters

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }

search = GridSearchCV(SVC(), param_grid)

search.fit(X_train,y_train)
clf_SVC = search.best_estimator_
pred= clf_SVC.predict(X_train)
acc = accuracy_score(y_train, pred)


f1=f1_score(y_train,pred,average = 'micro')

print ('F1 Score:{}'.format(f1))

print ('Acurácia LogisticRegression:{}'.format(acc))

print(clf_SVC)

In [ ]:
#Treinando e testando os modelos
print ('Decision Tree')


clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
pred= clf.predict(X_train)

dt_acc = accuracy_score(y_train, pred)
f1=f1_score(y_train,pred, average='macro')
print ('Acurácia Tree:{}'.format(dt_acc))
print ('F1 Score:{}'.format(f1) )


n_estimators = [10, 50, 100, 200]
max_depth = [3, 10, 20, 40]

In [ ]:
#Treinando e testando os modelos
print ('Naive baeys')


clf = GaussianNB()
clf.fit(X_train, y_train)
pred= clf.predict(X_train)

nb_acc = accuracy_score(y_train, pred)
f1=f1_score(y_train,pred, average='micro')
print ('Acurácia Naive baeys:{}'.format(nb_acc))
print ('F1 Score:{}'.format(f1) )

In [ ]:
#Executando a previsao

previsao=features_scale[2650:]

game_id_full=df['partida_id']
game_id=game_id_full[2650:]

res_full=df['resultado']
res=res_full[2650:]


pred=clf_LR.predict(previsao)

df=pd.DataFrame({'real': res, 'previsao':pred, 'partida_id':game_id})

print(df)

In [ ]:
#confusion Matrix

df=pd.DataFrame(df,columns=['real','previsao' ])

cf_matrix=pd.crosstab(df['real'], df['previsao'], rownames=['real'] , colnames=['previsao'])

sns.heatmap(cf_matrix, annot=True, cmap='Reds')

In [ ]:
# relatório do modelo
print('Relatório de classificação:\n', classification_report(y_train, y_pred, digits=4))

In [ ]:
# Functions for plotting
def DistributionPlot(RedFunction, BlueFunction, RedName, BlueName, Title):
    width = 12
    height = 10
    plt.figure(figsize=(width, height))

    ax1 = sns.distplot(RedFunction, hist=False, color="r", label=RedName)
    ax2 = sns.distplot(BlueFunction, hist=False, color="b", label=BlueName, ax=ax1)

    plt.title(Title)
    plt.xlabel('Price (in dollars)')
    plt.ylabel('Proportion of Cars')

    plt.show()
    plt.close()

In [ ]:
# Let's examine the distribution of the predicted values of the training data
Title = 'Distribution  Plot of  Predicted Value Using Training Data vs Training Data Distribution'
DistributionPlot(y_train, yhat_train, "Actual Values (Train)", "Predicted Values (Train)", Title)

In [ ]:


# Let's examine the distribution of the predicted values of the training data
Title = 'Distribution  Plot of  Predicted Value Using Training Data vs Training Data Distribution'
DistributionPlot(y_test, yhat_teste, "Actual Values (teste)", "Predicted Values (teste)", Title)
